In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.datasets import load_boston

data = load_boston()
# (['data', 'target', 'feature_names', 'DESCR', 'filename'])
print(data.keys())
print(data.feature_names)
# print(data.DESCR)
# target : 집값의 MEDV(중간값) 구하기

# 1. price 가격의 구성 확인

X = data.data
y = data.target   # 집값  #11.9 $,000

# print(y)

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# %matplotlib inline

df = pd.DataFrame(data = data.data, columns=data.feature_names)
df['price'] = data.target
print(df.shape)   #(506, 14)

(506, 14)


In [3]:
#----------------EDA

In [6]:
#----------------학습데이터 준비 7:3 -----------
from sklearn.model_selection import train_test_split
y = df['price']
X = df.drop(['price'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11, test_size=0.2)
# train_test_split(data.data, data.target, random_state=11, test_size=0.2)


In [7]:
# ----------------모델 선택 & 학습 & 예측답안 생성
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

model_lr = LinearRegression()
model_rf = RandomForestRegressor()

model_lr.fit(X_train,y_train)

y_pred = model_lr.predict(X_test)


In [8]:
#-----------------검증---
# (차이점) 타이타닉은 얼마나 맞췄는지 but!!회귀는 직선부터 얼마나 떨어져있는게 중요! 기울기 회귀계수 중요!!

# -----------직선방정식(회귀계수 = 기울기 = 직선)으로부터 각 점들(실제y정답지)간의 거리를 계산해야 함
# -------RSS: y-y^
# MSE 계산
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred,squared=True)
rmse = mean_squared_error(y_test, y_pred,squared=False)
print("mse:", mse)
print("rmse:", rmse)

mse: 26.944812249518307
rmse: 5.190839262539181


In [9]:
#KFold
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=11)

mse_list = []
rmse_list = []

for idx_train, idx_test in enumerate(kf.split(X)):
    X_train, X_test = X[idx_train], X[idx_test]
    y_train, y_test = y[idx_train], y[idx_test]
#-------이하 코드 동일 -----
for i in range(10):
    model_lr.fit(X_train, y_train)
    y_pred = model_lr.predict(X_test)
    mse = mean_squared_error(y_test, y_pred, squared=True)
    # rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(k, mse, rmse)
    mse_list.append(mse)
    # rmse_list.append(rmse)

#print
print("5회 MSE 평균:", np.mean(mse_list))
print("5회 RMSE 평균:", np.sqrt(np.mean(mse_list)))


c:\aip\python_workspace\venv\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


KeyError: 0

In [10]:
# cross_val_score
from sklearn.model_selection import cross_val_score
score_arr5 = cross_val_score(model_lr, X, y, scoring='neg_mean_squared_error', cv=5)
print("5회 mse:", score_arr5)
print("5회 mse 평균:", np.mean(score_arr5))
print("5회 rmse 평균:", np.sqrt(np.mean(score_arr5)))


5회 mse: [-12.46030057 -26.04862111 -33.07413798 -80.76237112 -33.31360656]
5회 mse 평균: -37.13180746769924
5회 rmse 평균: nan


c:\aip\python_workspace\venv\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  


In [14]:
from sklearn.model_selection import GridSearchCV
hyper_param = {'max_depth': [2,4,6,8], 'min_samples_leaf':[1,2,3,4,5]}
model_gcv = GridSearchCV(model_rf, scoring='neg_mean_squared_error', cv=5, param_grid=hyper_param, refit=True, verbose=0)
model_gcv.fit(X_train, y_train)
print("최고파라미터:", model_gcv.best_params_)
print("최고MSE:",model_gcv.best_score_)
print("최고모델:",model_gcv.best_estimator_)
y_pred = model_gcv.predict(X_test)
mse = mean_squared_error(y_test, y_pred, squared=True)
print(np.mean(mse))

최고파라미터: {'max_depth': 8, 'min_samples_leaf': 1}
최고MSE: -11.526423030408726
최고모델: RandomForestRegressor(max_depth=8)
18.547111299072327
